In [1]:
# %pip install scikit-learn


In [2]:
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score

In [76]:
df_99_22 = pd.read_csv('NBA_data//nba_games_1999_2022.csv', index_col=0)
df_23 = pd.read_csv('NBA_data//nba_games_2023.csv', index_col=0)

In [77]:
df = pd.concat([df_99_22, df_23])

In [78]:
df = df.sort_values('date')

In [79]:
df = df.reset_index(drop=True)

In [80]:
del df['index_opp']

In [81]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

In [82]:
df = df.groupby('team', group_keys=False).apply(add_target)

In [10]:
df[df['team'] == 'WAS']

,stl%,blk%,blk,3pa,3par,orb,tov%,3p,ast,ortg,...,fta_opp,3p%_opp,ts%_opp,team_opp,total_opp,home_opp,season,date,won,target
21,4.4,5.2,3,12,0.143,6,11.1,2,11,89.9,...,43,0.300,0.552,IND,96,1,1999,1999-02-05,False,True
24,10.7,0.0,0,14,0.169,15,13.6,5,24,105.1,...,22,0.545,0.547,TOR,97,0,1999,1999-02-06,True,False
63,8.8,0.0,0,11,0.138,15,15.5,5,25,113.3,...,48,0.267,0.588,DET,106,1,1999,1999-02-08,False,False
91,7.8,7.4,4,12,0.148,17,11.8,2,14,97.8,...,33,0.583,0.627,NYK,101,1,1999,1999-02-10,False,False
189,9.6,8.7,6,13,0.157,16,19.9,4,20,90.6,...,32,0.308,0.500,ORL,96,1,1999,1999-02-17,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62794,9.1,8.9,5,37,0.468,5,17.2,16,26,110.8,...,18,0.406,0.615,NYK,118,1,2023,2023-04-02,False,False
62818,6.4,8.0,4,27,0.273,9,7.7,13,30,117.9,...,29,0.548,0.668,MIL,140,0,2023,2023-04-04,False,False
62835,9.6,16.7,9,28,0.298,12,11.7,6,23,111.6,...,40,0.433,0.659,ATL,134,1,2023,2023-04-05,False,True
62866,3.9,5.9,3,33,0.393,12,13.8,7,23,111.3,...,12,0.319,0.523,MIA,108,0,2023,2023-04-07,True,False


In [85]:
df['target'][pd.isnull(df['target'])] = 2

C:\Users\USER\AppData\Local\Temp\ipykernel_16632\430200610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][pd.isnull(df['target'])] = 2


In [87]:
df

,stl%,blk%,blk,3pa,3par,orb,tov%,3p,ast,ortg,...,fta_opp,3p%_opp,ts%_opp,team_opp,total_opp,home_opp,season,date,won,target
0,12.2,0.0,0,3,0.038,13,15.7,1,15,91.7,...,43,0.200,0.556,ATL,100,1,1999,1999-02-05,False,0
1,12.4,5.7,3,13,0.153,15,11.2,6,20,108.3,...,42,0.500,0.671,UTA,104,1,1999,1999-02-05,False,1
2,6.0,6.1,4,22,0.227,20,5.5,6,24,109.7,...,15,0.333,0.475,DAL,86,0,1999,1999-02-05,True,1
3,2.4,5.3,4,18,0.214,12,9.0,6,21,102.5,...,13,0.273,0.448,SEA,92,1,1999,1999-02-05,False,1
4,14.2,7.5,6,12,0.141,18,18.4,4,24,102.5,...,20,0.250,0.396,SAC,83,0,1999,1999-02-05,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52957,8.1,4.7,2,28,0.373,9,13.3,11,23,124.3,...,20,0.486,0.639,MIA,111,0,2023,2023-06-04,False,1
52958,3.3,8.8,5,18,0.225,13,12.4,5,28,120.4,...,19,0.314,0.468,MIA,94,1,2023,2023-06-07,True,1
52959,7.7,4.8,3,35,0.380,10,3.8,11,20,103.8,...,27,0.278,0.593,DEN,109,0,2023,2023-06-07,False,0
52960,12.4,13.2,7,28,0.354,5,6.4,14,26,121.9,...,20,0.320,0.547,MIA,95,1,2023,2023-06-09,True,2


In [86]:
df['target'] = df['target'].astype(int, errors='ignore')

In [100]:
df['target'].value_counts()

1    26464
0    26461
2       37
Name: target, dtype: int64

In [14]:
nulls = pd.isnull(df)

In [15]:
nulls

,stl%,blk%,blk,3pa,3par,orb,tov%,3p,ast,ortg,...,fta_opp,3p%_opp,ts%_opp,team_opp,total_opp,home_opp,season,date,won,target
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63067,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63068,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63069,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63070,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
nulls = nulls.sum()

In [17]:
nulls = nulls[nulls > 0]

In [18]:
nulls

Series([], dtype: int64)

In [101]:
rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=15, direction='forward',cv=split)

In [102]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [103]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [104]:
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [105]:
sfs.fit(df[selected_columns], df['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=15)

In [136]:
predictors = list(selected_columns[sfs.get_support()])

In [107]:
predictors

['pts',
 'orb%',
 'fg%',
 'usg%',
 '+/-',
 '3p%',
 'ts%',
 'total',
 'ast_opp',
 'pts_opp',
 'drb%_opp',
 'ft%_opp',
 'usg%_opp',
 '3p%_opp',
 'total_opp']

In [108]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data['season'].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = data[data['season'] < season]
        test = data[data['season'] == season]

        model.fit(train[predictors], train['target'])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)

        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ['actual', 'prediction']

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [109]:
predictions = backtest(df, rr, predictors)

In [110]:
predictions

,actual,prediction
4124,1,1
4125,0,0
4126,0,1
4127,0,0
4128,1,0
...,...,...
52957,1,1
52958,1,1
52959,0,0
52960,2,1


In [111]:
predictions = predictions[predictions['actual'] != 2]
accuracy_score(predictions['actual'], predictions['prediction'])

0.5445157060016802

In [112]:
df.groupby('home').apply(lambda x: x[x['won'] == 1].shape[0] / x.shape[0])

home
0.0    0.408935
1.0    0.591065
dtype: float64

In [113]:
df_rolling = df[list(selected_columns) + ['won', 'team', 'season']]

In [114]:
df_rolling

,stl%,blk%,blk,3pa,3par,orb,tov%,3p,ast,ortg,...,+/-_opp,efg%_opp,fta_opp,3p%_opp,ts%_opp,total_opp,home_opp,won,team,season
0,0.514768,0.000000,0.00,0.028986,0.038235,0.342105,0.548507,0.034483,0.239130,0.367041,...,0.0,0.436720,0.666667,0.200,0.522727,0.401575,1.0,False,CLE,1999
1,0.523207,0.141089,0.15,0.173913,0.207353,0.394737,0.380597,0.206897,0.347826,0.522472,...,0.0,0.655971,0.650794,0.500,0.740530,0.433071,1.0,False,CHI,1999
2,0.253165,0.150990,0.20,0.304348,0.316176,0.526316,0.167910,0.206897,0.434783,0.535581,...,0.0,0.367201,0.222222,0.333,0.369318,0.291339,0.0,True,SEA,1999
3,0.101266,0.131188,0.20,0.246377,0.297059,0.315789,0.298507,0.206897,0.369565,0.468165,...,0.0,0.354724,0.190476,0.273,0.318182,0.338583,1.0,False,DAL,1999
4,0.599156,0.185644,0.30,0.159420,0.189706,0.473684,0.649254,0.137931,0.434783,0.468165,...,0.0,0.269162,0.301587,0.250,0.219697,0.267717,0.0,True,SAS,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52957,0.341772,0.116337,0.10,0.391304,0.530882,0.236842,0.458955,0.379310,0.413043,0.672285,...,0.0,0.645276,0.301587,0.486,0.679924,0.488189,0.0,False,DEN,2023
52958,0.139241,0.217822,0.25,0.246377,0.313235,0.342105,0.425373,0.172414,0.521739,0.635768,...,0.0,0.347594,0.285714,0.314,0.356061,0.354331,1.0,True,DEN,2023
52959,0.324895,0.118812,0.15,0.492754,0.541176,0.263158,0.104478,0.379310,0.347826,0.480337,...,0.0,0.552585,0.412698,0.278,0.592803,0.472441,0.0,False,MIA,2023
52960,0.523207,0.326733,0.35,0.391304,0.502941,0.131579,0.201493,0.482759,0.478261,0.649813,...,0.0,0.474153,0.301587,0.320,0.505682,0.362205,1.0,True,DEN,2023


In [115]:
def find_team_averages(team):
    rolling = team.rolling(20).mean()
    return rolling

In [116]:
df_rolling = df_rolling.groupby(['team', 'season'], group_keys=False).apply(find_team_averages)

In [117]:
rolling_cols = [f'{col}_10' for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [118]:
df = df.dropna()

In [119]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

In [120]:
def add_col(df, col_name):
    return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))

In [121]:
df['home_next'] = add_col(df, 'home')
df['team_opp_next'] = add_col(df, 'team_opp')
df['date_next'] = add_col(df, 'date')

In [122]:
df1 = df.copy()

In [123]:
full = df1.merge(
    df1[rolling_cols + ['team_opp_next', 'date_next', 'team']],
    left_on=['team', 'date_next'],
    right_on=['team_opp_next', 'date_next']
)

In [124]:
full[['team_x', 'team_opp_next_x', 'team_y', 'team_opp_next_y', 'date_next']]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,DAL,ORL,ORL,DAL,1999-03-11
1,ORL,DAL,DAL,ORL,1999-03-11
2,DEN,SAS,SAS,DEN,1999-03-13
3,DAL,VAN,VAN,DAL,1999-03-13
4,ORL,GSW,GSW,ORL,1999-03-13
...,...,...,...,...,...
40633,DEN,MIA,MIA,DEN,2023-06-04
40634,MIA,DEN,DEN,MIA,2023-06-07
40635,DEN,MIA,MIA,DEN,2023-06-07
40636,DEN,MIA,MIA,DEN,2023-06-09


In [125]:
removed_columns = list(full.columns[full.dtypes == 'object']) + removed_columns

In [126]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_opp_next_x',
 'date_next',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [127]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [128]:
sfs.fit(full[selected_columns], full['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=15)

In [129]:
predictors = list(selected_columns[sfs.get_support()])

In [130]:
predictors

['stl%',
 'mp',
 'usg%',
 '+/-',
 'mp_opp',
 'fg_opp',
 'usg%_opp',
 '+/-_opp',
 '3p%_opp',
 'ts%_opp',
 'ft%_10_x',
 'won_10_x',
 'home_next',
 'orb_opp_10_y',
 'won_10_y']

In [131]:
predictions = backtest(full, rr, predictors)

In [132]:
accuracy_score(predictions['actual'], predictions['prediction'])

0.6550286745964317

In [134]:
df1.groupby('home').apply(lambda x: x[x['won'] == 1].shape[0] / x.shape[0])

home
0.0    0.408570
1.0    0.591997
dtype: float64

In [135]:
df

,stl%,blk%,blk,3pa,3par,orb,tov%,3p,ast,ortg,...,fta_opp_10,3p%_opp_10,ts%_opp_10,total_opp_10,home_opp_10,won_10,season_10,home_next,team_opp_next,date_next
503,0.459916,0.274752,0.40,0.072464,0.107353,0.421053,0.626866,0.034483,0.195652,0.563670,...,0.456349,0.31305,0.411080,0.350394,0.50,0.70,1999.0,1.0,LAC,1999-03-10
508,0.248945,0.136139,0.20,0.202899,0.264706,0.315789,0.380597,0.137931,0.478261,0.521536,...,0.372222,0.32640,0.433333,0.358268,0.60,0.30,1999.0,1.0,ORL,1999-03-11
516,0.527426,0.111386,0.15,0.159420,0.217647,0.184211,0.511194,0.137931,0.304348,0.351124,...,0.293651,0.36875,0.441098,0.307480,0.45,0.70,1999.0,0.0,DAL,1999-03-11
529,0.421941,0.202970,0.25,0.043478,0.057353,0.473684,0.615672,0.034483,0.282609,0.482210,...,0.458730,0.32135,0.408239,0.344094,0.50,0.70,1999.0,1.0,GSW,1999-03-12
531,0.409283,0.081683,0.10,0.173913,0.236765,0.315789,0.533582,0.068966,0.347826,0.515918,...,0.432540,0.41140,0.516477,0.403150,0.55,0.25,1999.0,0.0,SAS,1999-03-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52957,0.341772,0.116337,0.10,0.391304,0.530882,0.236842,0.458955,0.379310,0.413043,0.672285,...,0.322222,0.35220,0.548485,0.463386,0.45,0.70,2023.0,0.0,MIA,2023-06-07
52958,0.139241,0.217822,0.25,0.246377,0.313235,0.342105,0.425373,0.172414,0.521739,0.635768,...,0.317460,0.34430,0.530208,0.453543,0.45,0.75,2023.0,0.0,MIA,2023-06-09
52959,0.324895,0.118812,0.15,0.492754,0.541176,0.263158,0.104478,0.379310,0.347826,0.480337,...,0.361905,0.32615,0.552936,0.455906,0.55,0.60,2023.0,1.0,DEN,2023-06-09
52960,0.523207,0.326733,0.35,0.391304,0.502941,0.131579,0.201493,0.482759,0.478261,0.649813,...,0.315079,0.34460,0.531250,0.444488,0.45,0.80,2023.0,NaN,NaN,NaN
